<a href="https://colab.research.google.com/github/RodolfoFerro/unet-workshop/blob/main/U_Net_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementación de U-Net con TensorFlow

> Contenido creado por **Rodolfo Ferro** ([Future Lab](https://futurelab.mx/), 2023 _–actualizado_). <br>
> Contacto: ig - [@rodo_ferro](https://www.instagram.com/rodo_ferro/) & tw - [@rodo_ferro](https://twitter.com/rodo_ferro)

En este cuaderno podrás encontrar un código base que implementa el modelo de U-Net para realizar un entrenamiento e inferencia de imágenes médicas.

## Clonamos el repositorio

Comenzaremos clonando el repositorio y asignando a la carpeta como la raíz.

In [ ]:
!git clone https://github.com/RodolfoFerro/unet-workshop.git
%cd unet-workshop
!ls

La estructura del código fuente es como sigue:
- `unet/model.py` - Contiene la implementación del U-Net.
- `utils/data.py` - Contiene funciones de utilería para carga de datos.
- `utils/image.py` - Coniene funciones de utilería para cargar imágenes y mostrar los resultados de las inferencias.
- `main.py` - Contiene una sencilla implementación de este cuaderno en un script de Python para entrenar el modelo.

------

## Cargamos los datos

A continuación procedemos a importar algunas bibliotecas y el código base del modelo.

Haremos uso de alunas funciones que permiten cargar datos que encuentras en el folder `data`.

Comenzaremos importando las funciones de los módulos a utilizar.

In [ ]:
from utils.data import train_generator
from utils.data import test_generator
from utils.data import save_results

Procedemos a crear un diccionario de configuración para cargar datos.

In [ ]:
data_gen_args = dict(
    rotation_range=0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = train_generator(
    2, 'data/membrane/train',
    'image', 'label',
    data_gen_args,
    save_to_dir=None
)

------

## Creamos el modelo

Ahora, procederemos a crear el modelo. Para ello, dos opciones serán previstas.

**OPCIÓN A:** Creamos nuestro propio U-Net con nuestras propias características, basándonos en la propuesta original:

<center>
    <img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-07-07_at_9.08.00_PM_rpNArED.png" width="60%">
</center>

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam


def unet(pretrained_weights=None, input_size=(256, 256, 1)):
    """U-Net model constructor.

    Parameters
    ----------
    pretrained_weights : str
        Path to pretrained weights.
    input_size : tuple
        Spatial size of the expected input image.
    """

    inputs = Input(input_size)

    # Convolution chain #1
    # conv_1 = ...
    
    # Continua aquí con tu propia implementación...

**OPCIÓN B:** Creamos una instancia del modelo ya implementado y entrenamos con los datos.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

from unet.model import unet


model = unet()
model_checkpoint = ModelCheckpoint(
    'unet_membrane.hdf5',
    monitor='loss',
    verbose=1,
    save_best_only=True
)

model.fit(
    train_gen,
    steps_per_epoch=300,
    epochs=5,
    callbacks=[model_checkpoint]
)

**¡Felicidades! Hasta este punto deberías haber entrenado exitosamente un U-Net con algunas imágenes médicas.**

Una vez entrenado el modelo, podemos realizar pruebas de inferencia con el conjunto de pruebas que se encuentra en la misma carpeta de datos.

In [ ]:
test_gen = test_generator('data/membrane/test')
results = model.predict(test_gen, 30, verbose=True)
save_results('data/results', results)

------

## Resultados gráficos

El código base provee algunas funciones para cargar, inferir y crear máscaras de los resultados al trabajar sobre algunas imágenes.

Procedemos a importar las funciones del módulo de imágenes.

In [ ]:
from utils.image import load_test_image
from utils.image import inference_over_image
from utils.image import create_mask
from utils.image import overlay_mask

Cargamos una imagen del directorio de prueba, especificando con un número entero el índice de alguna de las 30 imágenes (`[0, 29]`).

In [ ]:
img = load_test_image(0)

Usamos el modelo previamente entrenado para inferir sobre la imagen previamente cargada.

In [ ]:
out = inference_over_image(model, img)

Creamos una máscara a partir de la inferencia.

In [ ]:
mask = create_mask(out)

Sobreponemos la máscara en la imágen original para validar el resulatdo.

In [ ]:
res = overlay_mask(img, mask)

**¡Felicidades! Has utilizado exitosamente tu modelo entrenado sobre algunas imágenes médicas.**

**Reto:** Yo me he encargado de enfocarme en utilizar las detecciones para la identificación de pared celular, sin embargo, puedes modificar o crear tus propias funciones para la detección celular completa.

Por otro lado, no debes limitarte a ello, sino que puedes crear o cargar tu propio conjunto de datos para segmentar otro tipo de elementos, como las mitocondrias (echa un vistazo al [Electron Microscopy Dataset](https://www.epfl.ch/labs/cvlab/data/data-em/)).